In [46]:
import pandas as pd

#### Nuotekos

In [70]:
nuotekos = pd.read_csv("csv/nuotekos.csv")
nuotekos = nuotekos.replace("Klaipėdos m. sav.", "Klaipėda").replace("Kauno m. sav.", "Kaunas").replace("Vilniaus m. sav.", "Vilnius").replace("Panevėžio m. sav.", "Panevėžys").replace("Šiaulių m. sav.", "Šiauliai").replace("Alytaus m. sav.", "Alytus")
nuotekos = nuotekos.drop(['Rodiklis', "Matavimo vienetai"], 1)
nuotekos = pd.merge(
    nuotekos[nuotekos["Išvalymas"] == "Išleista išvalytų iki normos nuotekų"],
    nuotekos[nuotekos["Išvalymas"] == "Iš viso išleista nuotekų"],
    on=["Administracinė teritorija", "Laikotarpis"])
nuotekos["y"] = round(100*nuotekos["Reikšmė_x"]/nuotekos["Reikšmė_y"],2)
nuotekos = nuotekos.drop(['Išvalymas_x', "Išvalymas_y", "Reikšmė_x", "Reikšmė_y"], 1)
nuotekos = nuotekos.rename({'Laikotarpis': 'x', 'Administracinė teritorija': 'city'}, axis=1)
nuotekos.to_csv("kpi/nuotekos.csv", index = False)

#### Metalo laužas

In [7]:
mlauzas = pd.read_csv("csv/metalo_lauzas.csv")
mlauzas = mlauzas[(mlauzas["Supirkimo šaltinis"] == "Supirkta, iš viso")]
mlauzas = mlauzas.drop(['Rodiklis', "Matavimo vienetai", "Supirkimo šaltinis"], 1)
mlauzas = pd.merge(
    mlauzas[mlauzas["Metalų laužo rūšis"] == "Juodųjų metalų laužas, tūkst. tonų"],
    mlauzas[mlauzas["Metalų laužo rūšis"] == "Spalvotųjų metalų laužas, tonos"],
    on=["Apskritys", "Laikotarpis"])
mlauzas["y"] = mlauzas["Reikšmė_x"] + mlauzas["Reikšmė_y"]
mlauzas = mlauzas.drop(['Metalų laužo rūšis_x', "Metalų laužo rūšis_y", "Reikšmė_x", "Reikšmė_y"], 1)
mlauzas = mlauzas.rename({'Laikotarpis': 'x', 'Apskritys': 'city'}, axis=1)
mlauzas.to_csv("kpi/mlauzas.csv", index = False)

In [175]:
cat = pd.read_csv("kpi/categories.csv")
weights = [25, 25, 25, 25]

cat.iloc[:,[2,3,4,5]] = cat.iloc[:,[2,3,4,5]] * pd.Series(weights,index = cat.columns[[2,3,4,5]]) / 25 * 100 / sum(weights)
cat["total"]= cat.iloc[:,[2,3,4,5]].sum(axis=1)

vanduo = cat[["city","vanduo","x"]]
vanduo = vanduo.rename({'vanduo': 'y'}, axis=1)
vanduo.to_csv("kpi/vanduo.csv", index = False)

transportas = cat[["city","transportas","x"]]
transportas = transportas.rename({'transportas': 'y'}, axis=1)
transportas.to_csv("kpi/transportas.csv", index = False)

oras = cat[["city","oras","x"]]
oras = oras.rename({'oras': 'y'}, axis=1)
oras.to_csv("kpi/oras.csv", index = False)

siuksles = cat[["city","šiukšles","x"]]
siuksles = siuksles.rename({'šiukšles': 'y'}, axis=1)
siuksles = siuksles[siuksles["x"] < 2020]
siuksles.to_csv("kpi/siuksles.csv", index = False)

total = cat[["city","total","x"]]
total = total.rename({'total': 'y'}, axis=1)
total.to_csv("kpi/total.csv", index = False)

#### KPI

In [189]:
total = pd.read_csv("kpi/categories.csv")
kpi = total[["x","city","total"]]
kpi = kpi.rename({'total': 'y'}, axis=1)
kpi.to_csv("kpi/kpi.csv", index = False)

#### catbarchart

In [182]:
catbar = pd.read_csv("kpi/categories.csv")
catbar["šiukšles"] = list(catbar[catbar["x"] == 2019]["šiukšles"])*6
catbar = catbar[catbar["x"] == 2020]
catbar = catbar[["city", "transportas", "vanduo", "oras", "šiukšles"]]
catbar.to_csv("kpi/lycatbar.csv", index = False)

#### Indicator scores

In [115]:
iscores = pd.read_csv("kpi/indicators_scores.csv")
iscores["siuksles_surinktos"] = list(iscores[iscores["x"] == 2019]["siuksles_surinktos"])*6
iscores = iscores[iscores["x"] == 2020]
cols = [col for col in iscores.columns if col not in ["city","x"]]
result = {}
for city in iscores.city:
    result[city] = [{col:list(iscores[iscores["city"] == city].loc[:,col])[0]} for col in cols]
result

{'Šiauliai': [{'kelioniu_kiekis': 7.2},
  {'nuotekos': 8.3},
  {'oro_tersalai_co': 5.1},
  {'oro_tersalai_kietosios': 0.0},
  {'oro_tersalai_no': 1.7},
  {'siuksles_surinktos': 25.0},
  {'vandens_sunaudojimas_buiciai': 0.0},
  {'vandens_sunaudojimas_energetikai': 0.0},
  {'viesojo_rida': 8.2}],
 'Kaunas': [{'kelioniu_kiekis': 7.3},
  {'nuotekos': 8.3},
  {'oro_tersalai_co': 3.5},
  {'oro_tersalai_kietosios': 0.6},
  {'oro_tersalai_no': 0.4},
  {'siuksles_surinktos': 20.4},
  {'vandens_sunaudojimas_buiciai': 0.4},
  {'vandens_sunaudojimas_energetikai': 4.5},
  {'viesojo_rida': 0.0}],
 'Panevėžys': [{'kelioniu_kiekis': 7.2},
  {'nuotekos': 8.3},
  {'oro_tersalai_co': 8.3},
  {'oro_tersalai_kietosios': 8.3},
  {'oro_tersalai_no': 3.3},
  {'siuksles_surinktos': 19.4},
  {'vandens_sunaudojimas_buiciai': 0.1},
  {'vandens_sunaudojimas_energetikai': 1.4},
  {'viesojo_rida': 7.7}],
 'Alytus': [{'kelioniu_kiekis': 12.5},
  {'nuotekos': 8.3},
  {'oro_tersalai_co': 4.5},
  {'oro_tersalai_kietosio

In [201]:

df = pd.read_csv("kpi/categories.csv")
weights = [0.2,0.6,0.3,0.3]
df.iloc[:,[2,3,4,5]] = df.iloc[:,[2,3,4,5]] * pd.Series(weights,index = df.columns[[2,3,4,5]]) / 25 * 100 / sum(weights)
df["total"]= df.iloc[:,[2,3,4,5]].sum(axis=1)

df_ranks = df.copy()
df_ranks.iloc[:,2:] = df_ranks.iloc[:,1:].groupby("x").rank(ascending=False)
df_ranks = df_ranks[df_ranks["x"] == 2020]
df_ranks = df_ranks[["city", "total"]]
df_ranks = df_ranks.rename({"total":"y"}, axis = 1)
df_ranks.to_csv("kpi/totalranks.csv")

In [3]:
import pandas as pd

In [5]:
iscores = pd.read_csv("kpi/indicators_scores.csv")
weights = [25, 25, 25, 25]
trans = [2,10]
water = [3,8,9]
air = [4,5,6]
trash = [7]
categories = [trans,water,air,trash]
for i,j in zip(categories,weights):
    iscores.iloc[:,i] = iscores.iloc[:,i] * j / 25 * 100 / sum(weights)
iscores

,city,x,kelioniu_kiekis,nuotekos,oro_tersalai_co,oro_tersalai_kietosios,oro_tersalai_no,siuksles_surinktos,vandens_sunaudojimas_buiciai,vandens_sunaudojimas_pozeminis,viesojo_rida
0,Šiauliai,2015,5.3,NaN,4.7,7.5,8.3,0.0,8.3,8.3,4.3
1,Kaunas,2015,8.4,NaN,2.5,5.8,5.5,15.8,7.8,7.8,12.5
2,Panevėžys,2015,3.3,NaN,0.0,0.0,3.9,1.3,8.2,8.2,2.6
3,Alytus,2015,0.0,NaN,4.9,2.5,7.6,18.4,8.1,8.1,0.0
4,Klaipėda,2015,10.6,NaN,1.9,1.7,0.0,7.9,0.0,0.0,7.5
5,Vilnius,2015,12.5,NaN,8.3,8.3,8.1,25.0,8.0,8.0,3.0
6,Šiauliai,2016,5.1,8.3,6.0,7.9,8.0,14.6,8.3,8.3,3.0
7,Kaunas,2016,8.8,8.3,2.8,6.9,5.9,10.4,7.8,7.8,12.5
8,Panevėžys,2016,5.0,8.3,0.0,0.0,2.5,10.4,8.2,8.2,4.5
9,Alytus,2016,0.0,8.3,6.4,5.6,8.0,25.0,8.1,8.1,0.0


In [9]:
iscores

,Unnamed: 0,city,x,kelioniu_kiekis,nuotekos,oro_tersalai_co,oro_tersalai_kietosios,oro_tersalai_no,siuksles_surinktos,vandens_sunaudojimas_buiciai,vandens_sunaudojimas_pozeminis,viesojo_rida
30,30,Šiauliai,2020,5.3,8.3,3.3,8.3,6.7,0.0,8.3,8.3,4.3
31,31,Kaunas,2020,5.2,8.3,4.9,7.8,7.9,5.4,8.0,8.0,12.5
32,32,Panevėžys,2020,5.3,8.3,0.0,0.0,5.0,5.4,8.2,8.2,4.8
33,33,Alytus,2020,0.0,8.3,3.8,5.0,6.7,10.7,8.1,8.1,0.0
34,34,Klaipėda,2020,9.7,1.2,1.2,2.2,0.0,5.4,0.0,0.0,7.2
35,35,Vilnius,2020,12.5,0.0,8.3,8.3,8.3,25.0,8.1,8.1,6.6


In [10]:
iscores = pd.read_csv("kpi/indicators_scores_2.csv")
iscores["siuksles_surinktos"] = list(iscores[iscores["x"] == 2019]["siuksles_surinktos"])*6
iscores = iscores[iscores["x"] == 2020]
cols = [col for col in iscores.columns if col not in ["city","x","Unnamed: 0"]]
result = {}
for city in iscores.city:
    result[city] = [{col:list(iscores[iscores["city"] == city].loc[:,col])[0]} for col in cols]
result

{'Šiauliai': [{'kelioniu_kiekis': 5.3},
  {'nuotekos': 8.3},
  {'oro_tersalai_co': 3.3},
  {'oro_tersalai_kietosios': 8.3},
  {'oro_tersalai_no': 6.7},
  {'siuksles_surinktos': 0.0},
  {'vandens_sunaudojimas_buiciai': 8.3},
  {'vandens_sunaudojimas_pozeminis': 8.3},
  {'viesojo_rida': 4.3}],
 'Kaunas': [{'kelioniu_kiekis': 5.2},
  {'nuotekos': 8.3},
  {'oro_tersalai_co': 4.9},
  {'oro_tersalai_kietosios': 7.8},
  {'oro_tersalai_no': 7.9},
  {'siuksles_surinktos': 5.4},
  {'vandens_sunaudojimas_buiciai': 8.0},
  {'vandens_sunaudojimas_pozeminis': 8.0},
  {'viesojo_rida': 12.5}],
 'Panevėžys': [{'kelioniu_kiekis': 5.3},
  {'nuotekos': 8.3},
  {'oro_tersalai_co': 0.0},
  {'oro_tersalai_kietosios': 0.0},
  {'oro_tersalai_no': 5.0},
  {'siuksles_surinktos': 5.4},
  {'vandens_sunaudojimas_buiciai': 8.2},
  {'vandens_sunaudojimas_pozeminis': 8.2},
  {'viesojo_rida': 4.8}],
 'Alytus': [{'kelioniu_kiekis': 0.0},
  {'nuotekos': 8.3},
  {'oro_tersalai_co': 3.8},
  {'oro_tersalai_kietosios': 5.0},